In [1]:
import networkx as nx
import re

# Part 1

Idea= Problem is finding the maximum path sum in a tree that represents the sequential actions taken per minute.

- create a DAG of actions per blueprint with networkx, get all paths of length 24 and calculate highest score

In [41]:
from dataclasses import dataclass
from enum import Enum

class Robot(Enum):
    ORE = 1
    CLAY = 2
    OBSIDIAN = 3
    GEODE = 4

ROBOT_TYPES = [Robot.ORE, Robot.CLAY, Robot.OBSIDIAN, Robot.GEODE]

r1 = re.compile(r'ore robot costs (?P<ore>\d+)')
r2 = re.compile(r'clay robot costs (?P<ore>\d+)')
r3 = re.compile(r'obsidian robot costs (?P<ore>\d+) ore and (?P<clay>\d+) clay')
r4 = re.compile(r'geode robot costs (?P<ore>\d+) ore and (?P<obsidian>\d+) obsidian')

@dataclass
class Blueprint:
    # cost only ore
    ore_robot_cost: int
    # cost only ore
    clay_robot_cost: int
    # cost ore and clay
    obsidian_robot_cost: tuple[int,int]
    # cost ore and obsidian
    geode_robot_cost: tuple[int,int]

    @staticmethod
    def parse(s:str):
        m1 = r1.search(s)
        m2 = r2.search(s)
        m3 = r3.search(s)
        m4 = r4.search(s)

        return Blueprint(
            ore_robot_cost=int(m1.group("ore")),
            clay_robot_cost=int(m2.group("ore")),
            obsidian_robot_cost=(int(m3.group("ore")), int(m3.group("clay"))),
            geode_robot_cost=(int(m4.group("ore")), int(m4.group("obsidian"))),
        )

@dataclass
class State:
    blueprint: Blueprint
    ore:int
    ore_robots: int
    clay: int
    clay_robots: int
    obsidian: int
    obsidian_robots: int
    geodes: int
    geode_robots: int

    @staticmethod
    def init_from_blueprint(bp: Blueprint):
        return State(
            blueprint= bp,
            ore=0,
            ore_robots=1,
            clay=0,
            clay_robots=0,
            obsidian=0,
            obsidian_robots=0,
            geodes=0,
            geode_robots=0
        )
            

    def can_build(self, robot: Robot):
        match robot:
            case(Robot.ORE): return self.ore >= self.blueprint.ore_robot_cost
            case(Robot.CLAY):  return self.clay >= self.blueprint.clay_robot_cost
            case(Robot.OBSIDIAN): 
                ore, clay = self.blueprint.obsidian_robot_cost
                return (self.ore >= ore and self.clay >= clay)
            case(Robot.GEODE): 
                ore, obs = self.blueprint.geode_robot_cost
                return (self.ore >= ore and self.obsidian >= obs)

    def possible_continuations(self):
        # to do 
        "should return a list of possible continuation states"
        return None

    def advance(self, minutes=1):
        for _ in range(minutes):
            self.ore += self.ore_robots
            self.clay += self.clay_robots
            self.obsidian += self.obsidian_robots
            self.geodes += self.geode_robots

In [10]:
s = State(0,1,0,1,0,1,0,1)
s.advance(3)
s

State(ore=3, ore_robots=1, clay=3, clay_robots=1, obsidian=3, obsidian_robots=1, geodes=3, geode_robots=1)

In [43]:
blueprints: list[Blueprint] = []

with open("input.txt", "r") as f:
    for line in f:
        blueprints.append(Blueprint.parse(line.rstrip()))

blueprints[1]


Blueprint(ore_robot_cost=2, clay_robot_cost=2, obsidian_robot_cost=(2, 15), geode_robot_cost=(2, 7))

In [44]:
starting_states = [State.init_from_blueprint(bp) for bp in blueprints]

starting_states[1]

State(blueprint=Blueprint(ore_robot_cost=2, clay_robot_cost=2, obsidian_robot_cost=(2, 15), geode_robot_cost=(2, 7)), ore=0, ore_robots=1, clay=0, clay_robots=0, obsidian=0, obsidian_robots=0, geodes=0, geode_robots=0)